In [85]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
#from dotenv import load_dotenv
import hvplot.pandas
from pathlib import Path
import seaborn as sns
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

## Reading our data into dataFrames
Data includes:
 * Latitude and Longitude of all Canada Provinces~ from [url](https://www.latlong.net/category/provinces-40-60.html).
 * Price range for houses in different provinces of Canada from excel file and avg household income per province using csv file.
 * Stock Prices data for *RioCan(REI-UN.TO)* Real Estate Investment Trust  & *Tricon Residential(TCN.TO)* a Canadian real estate company 

In [86]:
#get the latitude and longitude valuse for Canada provinces using url
lat_lon_df = pd.read_html("https://www.latlong.net/category/provinces-40-60.html")

In [87]:
#converting our list into dataFrame
lat_lon_df = pd.DataFrame(np.reshape(lat_lon_df, (10,3)), columns = ['Province', 'Latitude', 'Longitude'])
#remove the unwanted string from our columns
lat_lon_df['Province'] = lat_lon_df['Province'].str.replace(', Canada', '')
#display our latitude & longitude dataFrame
lat_lon_df

,Province,Latitude,Longitude
0,Saskatchewan Province,55.0,-106.0
1,Prince Edward Island,46.25,-63.0
2,Ontario,50.0,-85.0
3,Nova Scotia,45.0,-63.0
4,Alberta,55.0,-115.0
5,British Columbia,53.726669,-127.647621
6,Manitoba,56.415211,-98.739075
7,Newfoundland and Labrador,53.135509,-57.660435
8,New Brunswick,46.49839,-66.159668
9,Quebec Province,53.0,-70.0


In [88]:
# Using the read_csv function and Path module 
# create a absorbed_unit_df DataFrame by importing the 
#'absorbed-units-price-range-canada-provinces-for-all-dwelling-types-2019-2022-en.xlsx file' from the Resources folder
absorbed_unit_df = pd.read_excel(Path("""Resources/absorbed-units-price-range-canada-provinces-for-all-dwelling-types-2019-2022-en.xlsx"""), 
                                header = 1,
                                sheet_name =None)

#create dataFrame and import average household income for each province across years data
avg_income_df = pd.read_csv(Path('Resources/avg_household_income_canada.csv'))
#create dataFrame and import RioCan and Tricon Residential stock data
rei_df = pd.read_csv(Path('Resources/REI.UN.csv'))
tcn_df = pd.read_csv(Path('Resources/TCN.TO.csv'))

In [89]:
#function to fethch the Province names from dictionary of DataFrames-absorbed_unit_df 
def getProvince(absorbed_unit_df):
    province = []
    for key in absorbed_unit_df.keys():
        province.append(key)
    return province

In [90]:
canada_housing_data_df = pd.DataFrame()
for i in getProvince(absorbed_unit_df):
    #absorbed_unit_df[i].set_index(['Year','Province'], inplace = True)
    canada_housing_data_df = pd.concat([canada_housing_data_df,absorbed_unit_df[i]], axis = 0)

canada_housing_data_df.set_index('Year', inplace = True)
display(canada_housing_data_df.head())
display(canada_housing_data_df.tail())

,Province,Price Range low($),Price Range high($),Absorbed - Single,Absorbed - Semi,Absorbed - Row,Absorbed - Apartment and other
Year,,,,,,,
2019,Newfoundland and Labrador,149999.0,NaN,0,0,0,0
2019,Newfoundland and Labrador,150000.0,199999.0,0,1,0,0
2019,Newfoundland and Labrador,200000.0,249999.0,18,3,4,10
2019,Newfoundland and Labrador,250000.0,299999.0,54,34,36,1
2019,Newfoundland and Labrador,300000.0,349999.0,72,9,13,0


,Province,Price Range low($),Price Range high($),Absorbed - Single,Absorbed - Semi,Absorbed - Row,Absorbed - Apartment and other
Year,,,,,,,
2022,British Columbia,2850000.0,2899999.0,19,0,0,3
2022,British Columbia,2900000.0,2949999.0,29,2,0,2
2022,British Columbia,2950000.0,2999999.0,51,1,0,12
2022,British Columbia,3000000.0,3049999.0,30,0,0,2
2022,British Columbia,3050000.0,NaN,557,2,2,96


In [91]:
#relevant years for analysis
np.unique(canada_housing_data_df.index)

array([2019, 2020, 2021, 2022])

In [92]:
#Fill NaN valuse with zero
df1=canada_housing_data_df.fillna(value = 0)

In [93]:
#Drop rows where all Dwelling Type Columns{ Absorbed Single, Semi, row & other} contain zeros
canada_housing_data_df = canada_housing_data_df.loc[
    (canada_housing_data_df[['Absorbed - Single','Absorbed - Semi','Absorbed - Row','Absorbed - Apartment and other']] != 0)
    .any(axis = 1)
]

#find the average price range and drop columns Price Range low($) & Price Range high($)
canada_housing_data_df['Avg of Price Range']= canada_housing_data_df[['Price Range low($)', 'Price Range high($)']].mean(axis=1)
canada_housing_data_df.drop(['Price Range low($)', 'Price Range high($)'], axis = 1, inplace = True)


#disply sample data in the dataFrame
display(canada_housing_data_df.head())
display(canada_housing_data_df.tail())

,Province,Absorbed - Single,Absorbed - Semi,Absorbed - Row,Absorbed - Apartment and other,Avg of Price Range
Year,,,,,,
2019,Newfoundland and Labrador,0,1,0,0,174999.5
2019,Newfoundland and Labrador,18,3,4,10,224999.5
2019,Newfoundland and Labrador,54,34,36,1,274999.5
2019,Newfoundland and Labrador,72,9,13,0,324999.5
2019,Newfoundland and Labrador,41,0,5,0,374999.5


,Province,Absorbed - Single,Absorbed - Semi,Absorbed - Row,Absorbed - Apartment and other,Avg of Price Range
Year,,,,,,
2022,British Columbia,19,0,0,3,2874999.5
2022,British Columbia,29,2,0,2,2924999.5
2022,British Columbia,51,1,0,12,2974999.5
2022,British Columbia,30,0,0,2,3024999.5
2022,British Columbia,557,2,2,96,3050000.0


In [94]:
'''
#merge lat_lon_df & canada_housing_data_df to get the lat & long valuse in our dataFrame
canada_housing_data_df = canada_housing_data_df.merge(lat_lon_df, on = 'Province', how = 'inner' )

#merge avg_income_df & canada_housing_data_df to get the avg income valuse in our final dataFrame
canada_housing_data_df = canada_housing_data_df.merge(avg_income_df, on = 'Province', how = 'inner' )
'''

"\n#merge lat_lon_df & canada_housing_data_df to get the lat & long valuse in our dataFrame\ncanada_housing_data_df = canada_housing_data_df.merge(lat_lon_df, on = 'Province', how = 'inner' )\n\n#merge avg_income_df & canada_housing_data_df to get the avg income valuse in our final dataFrame\ncanada_housing_data_df = canada_housing_data_df.merge(avg_income_df, on = 'Province', how = 'inner' )\n"

In [95]:
#reorganize columns in canada_housing_data_df
canada_housing_data_df=canada_housing_data_df.reindex(sorted(canada_housing_data_df.columns, reverse = True), axis=1)

Questions:

Q1.trend across the year--> Province wise

Q2.which province suffered major impact-->  benefitted or sufered loss

Q3.most units sold and min units sold --> specific province and Year. --> the price range

Relation between inflation rate, avg income, unit sold and avg house price
(2-d at a time)

Q4.stock correlation with housing price in Canada housing price in Canada

Q5. Market Demand by Price Range: Which price range has the most absorbed units, indicating the highest demand in the market? How does this vary across different provinces?

Q6. Affordability Analysis: Based on the median household income in each province, which price ranges are realistically affordable to most families?

Q7. Income Requirements for Home Ownership: Based on the absorbed units’ price range in each province, what is the estimated annual income required to afford housing? How does this compare across provinces, and how does it align with the actual median annual incomes in those provinces?

In [96]:
# Rename the columns
new_column_names = {
    'Absorbed - Single': 'Single',
    'Absorbed - Semi': 'Semi',
    'Absorbed - Row': 'Row',
    'Absorbed - Apartment and other': 'Apartment and other',
}

canada_housing_data_df = canada_housing_data_df.rename(columns=new_column_names)
canada_housing_data_df.tail(5)

,Province,Avg of Price Range,Single,Semi,Row,Apartment and other
Year,,,,,,
2022,British Columbia,2874999.5,19,0,0,3
2022,British Columbia,2924999.5,29,2,0,2
2022,British Columbia,2974999.5,51,1,0,12
2022,British Columbia,3024999.5,30,0,0,2
2022,British Columbia,3050000.0,557,2,2,96


In [97]:
#calculate the total units sold in each provinces per year.
canada_housing_data_df['Total Absorbed Units']=canada_housing_data_df[["Single","Semi","Row","Apartment and other"]].sum(axis=1)
sum_of_sales=canada_housing_data_df.groupby(['Province','Year'])['Total Absorbed Units'].sum()
#sum_of_sales.hvplot.bar()

#Comment from ZHU:
#The purpose of this secetion of code is to show the overall volume in housing market.
#I dont think this graph is infomative enough. since the original data set is Absorbed Units, which is the newly built properties.
#Some provinces may have higher volume in terms of pre-owned houses. There may be provinces that don't build as many new houses.

weighted price for single house =

(# of unit from 1st price range in 2019/total unit in 2019) * midpoint price of 1st price range +

(# of unit from 2nd price range in 2019/total unit in 2019) * midpoint price of 2nd price range +

...

In [98]:
# Define dwelling types
dwelling_types = ['Single', 'Semi', 'Row', 'Apartment and other']

# Initialize an empty DataFrame
housing_price_df = pd.DataFrame()

# For Loop for each dwelling type
for dwelling_type in dwelling_types:
    # Calculate weighted price for each dwelling type
    canada_housing_data_df[f'weighted_price_{dwelling_type}'] = canada_housing_data_df[f'{dwelling_type}'] * canada_housing_data_df['Avg of Price Range']
    
    # Group by Province and Year and sum up the weighted price
    weighted_price_grouped = canada_housing_data_df.groupby(['Province', 'Year'])[f'weighted_price_{dwelling_type}'].sum()

    # Group by Province and Year and sum up the absorbed units
    absorbed_units_grouped = canada_housing_data_df.groupby(['Province', 'Year'])[f'{dwelling_type}'].sum()

    # Merge the two series into a DataFrame
    merged = pd.DataFrame(weighted_price_grouped)
    merged[f'{dwelling_type}'] = absorbed_units_grouped

    # Calculate the weighted average price
    merged[f'Weighted_Avg_Price_{dwelling_type}'] = merged[f'weighted_price_{dwelling_type}'] / merged[f'{dwelling_type}']

    # Drop the intermediate columns
    merged.drop([f'weighted_price_{dwelling_type}', f'{dwelling_type}'], axis=1, inplace=True)

    # Merge the result into the housing_price_df DataFrame
    if housing_price_df.empty:
        housing_price_df = merged
    else:
        housing_price_df = pd.merge(housing_price_df, merged, on=['Province', 'Year'])

housing_price_df = housing_price_df.reset_index()

In [99]:
housing_price_df.tail()

,Province,Year,Weighted_Avg_Price_Single,Weighted_Avg_Price_Semi,Weighted_Avg_Price_Row,Weighted_Avg_Price_Apartment and other
35,Quebec,2022,577037.187983,405320.470017,510538.991966,424006.628392
36,Saskatchewan,2019,479545.005747,360905.546980,327507.336991,345480.551020
37,Saskatchewan,2020,466820.664504,371153.354701,334920.134921,272958.683673
38,Saskatchewan,2021,472823.871475,385525.815789,362974.183544,453683.800000
39,Saskatchewan,2022,536170.250670,416070.928571,379048.082996,360923.882353


In [100]:
# Define a function to plot trend for each dwelling type
def plot_trend(dwelling_type):   
    # The melt function is used to convert the DataFrame from wide format to long format.
    plot_df = pd.melt(housing_price_df, id_vars=['Year', 'Province'], value_vars=[f'Weighted_Avg_Price_{dwelling_type}'])
    
    # Create a line plot of the WAP over years for each province using the hvplot library
    trend_plot=plot_df.hvplot.line('Year', 'value', by='Province',width=800).opts(yformatter='%.0f',title=f'Price trend for {dwelling_type}')
    
    return trend_plot

In [101]:
# Reset the index of the housing_price_df dataframe
housing_price_df.reset_index(inplace=True)

#Plot Price Trend for all the dwelling type
plot_trend('Single')+plot_trend('Semi')+plot_trend('Row')+plot_trend('Apartment and other')

:Layout
   .NdOverlay.I   :NdOverlay   [Province]
      :Curve   [Year]   (value)
   .NdOverlay.II  :NdOverlay   [Province]
      :Curve   [Year]   (value)
   .NdOverlay.III :NdOverlay   [Province]
      :Curve   [Year]   (value)
   .NdOverlay.IV  :NdOverlay   [Province]
      :Curve   [Year]   (value)

In [102]:
#Question2
#we are using Single house price as reference for return on housing market. 
return_house_df=housing_price_df[['Year', 'Province','Weighted_Avg_Price_Single']].copy()
return_house_df.set_index(['Year', 'Province'], inplace=True)
return_house_df['Annual Return %']=return_house_df.groupby('Province')['Weighted_Avg_Price_Single'].pct_change()
return_house_df['Annual Return %']=return_house_df['Annual Return %']*100

return_house_df.tail()

,,Weighted_Avg_Price_Single,Annual Return %
Year,Province,,
2022,Quebec,577037.187983,13.704914
2019,Saskatchewan,479545.005747,NaN
2020,Saskatchewan,466820.664504,-2.653420
2021,Saskatchewan,472823.871475,1.285977
2022,Saskatchewan,536170.250670,13.397458


In [103]:
CGR = ((return_house_df.loc[2022]['Weighted_Avg_Price_Single'] / return_house_df.loc[2019]['Weighted_Avg_Price_Single']) - 1)*100
CGR_df = pd.DataFrame(CGR).rename(columns={"Weighted_Avg_Price_Single": "Cumulative Return %"})
CGR_df.hvplot.bar(rot=30).opts(title="Cumulative growth for each province in the past 4 years")

:Bars   [Province]   (Cumulative Return %)

In [104]:
return_house_df['Annual Return %'].dropna().hvplot.bar(groupby='Year',rot=30,height=400)

:DynamicMap   [Year]
   :Bars   [Year,Province]   (Annual Return %)

In [109]:
#question3
#most units sold and min units sold --> specific province and Year. --> the price range
#Most popular type of house and price range in each province in each year
sales_house_df = canada_housing_data_df.reset_index()

sales_house_df = sales_house_df.melt(id_vars=['Year','Province', 'Avg of Price Range'], 
                                          value_vars=['Single', 'Semi', 'Row', 'Apartment and other'], 
                                          var_name='House Type', 
                                          value_name='Units Sold')

In [110]:
grouped_sales_house_df=sales_house_df.groupby(['Province', 'Year', 'House Type'])['Units Sold'].sum().reset_index()
grouped_sales_house_df.tail()

,Province,Year,House Type,Units Sold
155,Saskatchewan,2021,Single,1241
156,Saskatchewan,2022,Apartment and other,119
157,Saskatchewan,2022,Row,247
158,Saskatchewan,2022,Semi,140
159,Saskatchewan,2022,Single,1119


In [113]:
idx = grouped_sales_house_df.groupby(['Province', 'Year'])['Units Sold'].idxmax()
most_popular_type_df=grouped_sales_house_df.loc[idx]
most_popular_type_df.sample(5)
#visualization

,Province,Year,House Type,Units Sold
128,Quebec,2019,Apartment and other,6638
91,Nova Scotia,2021,Single,1052
147,Saskatchewan,2019,Single,957
83,Nova Scotia,2019,Single,804
155,Saskatchewan,2021,Single,1241


In [108]:
#question4


#correlation = rei_df_yearly['Return'].corr(return_house_df['Annual Return %'])
